<a href="https://colab.research.google.com/github/ricardomsanti/douExplore/blob/master/DEX_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import requests as r
!pip install html5lib
!pip install pyspark
!pip install PyPDF2
!pip install html5lib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# DOU

In [ ]:
## ACESSAR
# https://www.in.gov.br/servicos/diario-oficial-da-uniao
## IR PARA BASES DE DADOS E PUBLICAÇÕES // DIÁRIO COMPLETO: VERSÃO CERTIFICADA
## BAIXAR OS PDFS E COLOCAR NA PASTA CORRESPONDENTE EM 

## FEDERAL
"https://pesquisa.in.gov.br/imprensa/core/jornalList.action"

In [ ]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 11.7 MB/s eta 0:00:00


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 20.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=b0c6af92bc5c071ab0c3fa702fb03ff0049186afcd9ecf9e93c1a21d27c83ea6
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
import os
import PyPDF2
from PyPDF2 import PdfReader
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("my_app") \
    .getOrCreate()

In [ ]:
import string
from string import punctuation
import pyspark.sql.functions as F

FILES_PATH = "/content/drive/MyDrive/DATA/DOU/SHARE"

FILES = [[x.name, \
          x.path]\
           for x in os.scandir(FILES_PATH) if ".pdf" in str(x)]
FILES 
########################
text_lines = []
for f in FILES:
  obj = PdfReader(open(f[1], 'rb'))
  ext_line = []
  for page in range(0, len(obj.pages)):
    main_text = obj.pages[page].extract_text().split("\n")
    for line in range(0,len(main_text)):
      ext_line = []
      ext_line = [f[1], page, line, main_text[line]]
      print(f"Page >> {page}  \t" +
      f" Line >> {line}  \t" +
      f" Object Pages: >> {len(obj.pages)}")
      text_lines.append(ext_line)


lines = pd.DataFrame(text_lines)
lines.columns = ["path", "pg_num"," line", "string"]
data = spark.createDataFrame(lines)
data = data.withColumn("texto_regex", F.regexp_replace("string", "[\$#,\"!%&'()*+-./:;<=>?@^_`´{|}~\\\\]", ""))
data = data.withColumn("texto_trim", F.trim(data.texto_regex))

data.show(10, False)

In [ ]:
data.show(10, False)

+---------------------------------------------------------------------+------+-----+-------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------+
|path                                                                 |pg_num| line|string                                                                                                                               |texto_regex                                                                                                                    |texto_trim                                                                                                                     |
+-------------------------------------------------

In [ ]:
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol = "texto_trim", outputCol = "tokens")
tokenizado = tokenizer.transform(data)

tokenizado.show(10)


+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+
|                path|pg_num| line|              string|         texto_regex|          texto_trim|              tokens|
+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+
|/content/drive/My...|     0|    0|REPÚBLICA FEDERAT...|REPÚBLICA FEDERAT...|REPÚBLICA FEDERAT...|[república, feder...|
|/content/drive/My...|     0|    1|Ano LXIV Nº 13 Br...|Ano LXIV Nº 13 Br...|Ano LXIV Nº 13 Br...|[ano, lxiv, nº, 1...|
|/content/drive/My...|     0|    2|                   2|                   2|                   2|                 [2]|
|/content/drive/My...|     0|    3|Documento assinad...|Documento assinad...|Documento assinad...|[documento, assin...|
|/content/drive/My...|     0|    4|que institui a In...|que institui a In...|que institui a In...|[que, institui, a...|
|/content/drive/My...|     0|    5|http:

In [ ]:

from pyspark.sql.types import IntegerType

# Entendendo em quantos tokens os comentários foram divididos.
countTokens = F.udf(lambda tokens: len(tokens), IntegerType())

tokenizado.select("texto_trim", "tokens")\
          .withColumn("Freq_tokens", countTokens(F.col("tokens"))).show()

tokenizado.select("texto_trim", "tokens").show()

+--------------------+--------------------+-----------+
|          texto_trim|              tokens|Freq_tokens|
+--------------------+--------------------+-----------+
|REPÚBLICA FEDERAT...|[república, feder...|          7|
|Ano LXIV Nº 13 Br...|[ano, lxiv, nº, 1...|         14|
|                   2|                 [2]|          1|
|Documento assinad...|[documento, assin...|          9|
|que institui a In...|[que, institui, a...|         17|
|httpwwwingovbraut...|[httpwwwingovbrau...|          5|
|Presidência da Re...|[presidência, da,...|          5|
|Ministério da Agr...|[ministério, da, ...|          8|
|Ministério da Ciê...|[ministério, da, ...|          8|
|Ministério das Co...|[ministério, das,...|          5|
|Ministério da Def...|[ministério, da, ...|          5|
|Ministério do Des...|[ministério, do, ...|          6|
|Ministério da Eco...|[ministério, da, ...|          5|
|Ministério da Edu...|[ministério, da, ...|          3|
|                  17|                [17]|     

In [ ]:
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
stop_A = stopwords.words("portuguese")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Exemplo explorando bibliotecas. (pyspark)
from pyspark.ml.feature import StopWordsRemover

stop_B = StopWordsRemover.loadDefaultStopWords("portuguese")

In [ ]:
# exemplo comparando os diferentes stopwords #A
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol = "texto_trim", outputCol = "tokens")
tokenized = tokenizer.transform(data)

remover = StopWordsRemover(inputCol = "tokens", outputCol = "texto_final", stopWords= stop_A)
df = remover.transform(tokenized)

df.show(truncate = False)

+---------------------------------------------------------------------+------+-----+----------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------+
|path                                                                 |pg_num| line|string                                                                                                                                  |texto_regex                   

In [ ]:
# exemplo comparando os diferentes stopwords #B
remover = StopWordsRemover(inputCol = "tokens", outputCol = "texto_final", stopWords=stop_B)
df = remover.transform(tokenized)

df.show(truncate = False)

# 4. Retirando os stopwords dos depoimentos
remover = StopWordsRemover(inputCol = "tokens", outputCol = "texto_final")
feature_data = remover.transform(tokenizado)


# Contagem de tokens antes e depois do stopwords.
feature_data.select("tokens", "texto_final")\
          .withColumn("Freq_tokens", countTokens(f.col("tokens")))\
          .withColumn("Freq_tokens_limpos",  countTokens(f.col("texto_final"))).show()